In [1]:
from data.data_processing import data_processing
from data.data_transforms import data_transform_std
from data.data_splitting import make_input_output_sequences,train_test_split,shift_sequence
import pandas as pd
import torch
import numpy as np
import torch.utils.data as data_utils
from utils.metrics import MAE,SMAPE
from torch import nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

horizons = 4
test_ratio = 0.5
file_name = 'datasets/covid_weekly.csv'
data = pd.read_csv(file_name)
target = 'new_deaths'
features = data.columns
print(features.size)
m = ["RNN"]
batch_size =32
seq_len = 4
in_dim = len(features)-1

8


In [2]:
data, observed = data_processing(file_name, features,target)
scalers, df = data_transform_std(data, test_ratio)
x, y = make_input_output_sequences(data.values, seq_len, horizons, True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_ratio)


In [3]:
model = nn.RNN(features.size-1, 32, 4)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [4]:
for w in range(x_test.shape[0]):
    x_train_tensor = torch.from_numpy(np.array(x_train[:,:,1:], dtype=np.float32)).float().to(device)
    y_train_tensor = torch.from_numpy(np.array(y_train[:,:,1:], dtype=np.float32)).float().to(device)
    x_test_tensor = torch.from_numpy(np.array(x_test[:,:,1:], dtype=np.float32)).float().to(device)
    y_test_tensor= torch.from_numpy(np.array(y_test[:,:,1:], dtype=np.float32)).float().to(device)
    train = data_utils.TensorDataset(x_train_tensor, y_train_tensor)
    train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=False)
    
    for epoch in range(20):
        for x_t, y_t in train_loader:
            optimizer.zero_grad()
            y_pred, h_train = model(x_t)
            loss = criterion(y_pred[:,:,0], y_t[:,:,0])
            loss.backward()
            optimizer.step()
        # print(f'Epoch {epoch+1}/{20}, Loss: {loss.item()}')
y_test_pred, h_test = model(x_test_tensor)
print(type(y_test_pred))
norm_pred=scalers['scaler_new_deaths'].inverse_transform(y_test_pred[:,:,0].cpu().detach().numpy())
norm_true=scalers['scaler_new_deaths'].inverse_transform(y_test_tensor[:,:,0].cpu().detach().numpy())
smape = [SMAPE(norm_true[:,h],norm_pred[:,h]) for h in range(4)]
print(smape,np.mean(smape))

normMae= [MAE(norm_true[:,h],norm_pred[:,h]) for h in range(4)]
print(normMae, np.mean(normMae))

preds = y_test_pred[:,:,0].cpu().detach().numpy()
trues=y_test_tensor[:,:,0].cpu().detach().numpy()
Mae = [MAE(trues[:,h], preds[:,h]) for h in range(4)]
print(Mae, np.mean(Mae))

<class 'torch.Tensor'>
[49.746986671730326, 49.988438170633195, 52.278396229685086, 52.70282604076244] 51.17916177820276
[2817.594, 2753.7634, 2861.2854, 2859.8506] 2823.1233
[0.48050553, 0.46962008, 0.48795658, 0.48771188] 0.48144853


In [5]:
#ROLLING WINDOW for RNN
ListPred = []
ListTrue = []
normListPred = []
normListTrue = []
for w in range(x_test.shape[0]):
    x_train_tensor = torch.from_numpy(np.array(x_train[:,:,1:], dtype=np.float32)).float().to(device)
    y_train_tensor = torch.from_numpy(np.array(y_train[:,:,1:], dtype=np.float32)).float().to(device)
    x_test_tensor = torch.from_numpy(np.array(x_test[:,:,1:], dtype=np.float32)).float().to(device)
    y_test_tensor= torch.from_numpy(np.array(y_test[:,:,1:], dtype=np.float32)).float().to(device)
#     print(y_test_tensor)
    train = data_utils.TensorDataset(x_train_tensor, y_train_tensor)
    train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=False)

    for epoch in range(20):
        for x_t, y_t in train_loader:
            optimizer.zero_grad()
            y_pred, h_train = model(x_t)
            loss = criterion(y_pred[:,:,0], y_t[:,:,0])
            loss.backward()
            optimizer.step()
        y_test_pred, h_test = model(x_test_tensor)
        ListTrue.append(scalers['scaler_new_deaths'].inverse_transform(y_test_tensor[:, :, 0].cpu().detach().numpy())[0,:])
        ListPred.append(scalers['scaler_new_deaths'].inverse_transform(y_test_pred[:,:,0].cpu().detach().numpy())[0,:])
        normListTrue.append((y_test_tensor[:, :, 0].cpu().detach().numpy())[0,:])
        normListPred.append((y_test_pred[:,:,0].cpu().detach().numpy())[0,:])
    x_train, x_test, y_train, y_test = shift_sequence(x_train, y_train, x_test, y_test, 1, True)
ListTrue = np.array(ListTrue)
ListPred = np.array(ListPred)
normListTrue = np.array(normListTrue)
normListPred = np.array(normListPred)

smapes_RW=[SMAPE(ListTrue[:,h], ListPred[:,h]) for h in range(4)]
mae_RW=[MAE(ListTrue[:,h], ListPred[:,h]) for h in range(4)]
normMae_RW=[MAE(normListTrue[:,h], normListPred[:,h]) for h in range(4)]

print(smapes_RW,np.mean(smapes_RW))
print(mae_RW,np.mean(mae_RW))
print(normMae_RW,np.mean(normMae_RW))

[37.77957869164737, 39.618989920910494, 41.241952401620374, 41.63648440514082] 40.06925135482976
[1978.0157, 2037.4532, 2100.6936, 2128.2437] 2061.1016
[0.3373259, 0.3474623, 0.35824713, 0.36294544] 0.3514952
